In [1]:
# Принудительная перезагрузка модулей и импорты
import importlib
import sys

sys.path.append("src")

# Настройка логирования
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Импортируем модули с принудительной перезагрузкой
import src.classical_models
import src.cross_validation
import src.data_preparation
import src.model_comparison
import src.neural_models

# Перезагружаем модули для получения последних изменений
importlib.reload(src.classical_models)
importlib.reload(src.data_preparation)
importlib.reload(src.model_comparison)
importlib.reload(src.neural_models)
importlib.reload(src.cross_validation)

from src.classical_models import train_classical_models
from src.cross_validation import run_cross_validation
from src.data_preparation import prepare_data_pipeline
from src.model_comparison import create_model_ranking, create_performance_comparison
from src.neural_models import train_neural_models

logger.info("Модули успешно импортированы и перезагружены!")

INFO:rdkit:Enabling RDKit 2025.03.3 jupyter extensions
INFO:src:✅ Оптимизированные современные модели доступны
INFO:src:✅ Графовые утилиты доступны
INFO:src:📦 Инициализация пакета COX-2 Dataset Preparation v1.2.0
INFO:src:🚀 Оптимизированные модели готовы к использованию
INFO:src:📊 Графовые утилиты готовы к использованию
INFO:src:✅ Инициализация завершена
INFO:__main__:Модули успешно импортированы и перезагружены!


In [2]:
# Импорты
import sys

sys.path.append("src")

# Настройка логирования
import logging

import polars as pl

# Импортируем наши модули

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Модули успешно импортированы!")

INFO:__main__:Модули успешно импортированы!


In [3]:
# Подготовка данных
data_path = "data/descriptors/cox2_best_combined.parquet"
target_column = "pic50"

# Используем пайплайн подготовки данных
data_prepared = prepare_data_pipeline(
    file_path=data_path,
    target_column=target_column,
    test_size=0.2,
    variance_threshold=0.01,
    normalization="standard",
    random_state=42,
)

X_train = data_prepared["X_train"]
X_test = data_prepared["X_test"]
y_train = data_prepared["y_train"]
y_test = data_prepared["y_test"]

logger.info("Данные подготовлены:")
logger.info(f"Обучающая выборка: {X_train.shape}")
logger.info(f"Тестовая выборка: {X_test.shape}")
logger.info(f"Исходная форма: {data_prepared['original_shape']}")
logger.info(f"Финальная форма: {data_prepared['final_shape']}")

INFO:src.data_preparation:Загружены данные: 7322 строк, 712 колонок
INFO:src.data_preparation:Очищено данных: 7322 -> 3661 строк (3661 удалено)
INFO:src.data_preparation:Отфильтровано признаков: 711 -> 674 (порог дисперсии: 0.01)
INFO:src.data_preparation:Нормализованы признаки методом: standard
INFO:src.data_preparation:Данные разделены: обучение 2928, тест 733
INFO:__main__:Данные подготовлены:
INFO:__main__:Обучающая выборка: (2928, 674)
INFO:__main__:Тестовая выборка: (733, 674)
INFO:__main__:Исходная форма: (7322, 712)
INFO:__main__:Финальная форма: (3661, 674)


In [4]:
# === ОБОСНОВАНИЕ ВЫБОРА МОДЕЛЕЙ ===
logger.info("=== ОБОСНОВАНИЕ ВЫБОРА МОДЕЛЕЙ ===")
logger.info("1. Random Forest - ensemble-метод, устойчивый к переобучению, хорошо работает с молекулярными дескрипторами")
logger.info("2. XGBoost/Gradient Boosting - современный градиентный бустинг с высокой точностью")
logger.info("3. MLP - полносвязная нейросеть для изучения нелинейных паттернов")
logger.info("4. CNN - сверточная сеть для обнаружения локальных паттернов в дескрипторах")

# === ОБУЧЕНИЕ МОДЕЛЕЙ И КРОСС-ВАЛИДАЦИЯ ===
logger.info("\n=== ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА ОБУЧЕНИЯ МОДЕЛЕЙ ===\n")

# 1. Классические ensemble-модели (Random Forest, XGBoost/LightGBM)
logger.info("1. Обучение классических ensemble-моделей...")
classical_results = train_classical_models(X_train, y_train, X_test, y_test, random_state=42)

# 2. Нейросетевые модели с увеличенным числом эпох для лучшего обучения
logger.info("\n2. Обучение нейросетевых моделей...")
neural_results = train_neural_models(X_train, y_train, X_test, y_test, epochs=50, random_state=42)

# 3. Кросс-валидация для оценки стабильности
logger.info("\n3. Запуск кросс-валидации для анализа стабильности...")
cv_results = run_cross_validation(
    X_train,
    y_train,
    model_types=["random_forest", "xgboost", "mlp"],  # Выбираем основные модели для CV
    cv_folds=5,
    random_state=42,
)

# 4. Объединяем результаты
all_results = {**classical_results, **neural_results}

logger.info("\n=== РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ ===")
for model_name, metrics in all_results.items():
    logger.info(
        f"{model_name:15}: MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}, R²={metrics['r2']:.4f}, Время={metrics['training_time']:.2f}s"
    )

logger.info("\n=== РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ (стабильность) ===")
for model_name, cv_metrics in cv_results.items():
    mae_mean = cv_metrics["mae_mean"]
    mae_std = cv_metrics["mae_std"]
    r2_mean = cv_metrics["r2_mean"]
    r2_std = cv_metrics["r2_std"]
    logger.info(f"{model_name:15}: MAE={mae_mean:.4f}±{mae_std:.4f}, R²={r2_mean:.4f}±{r2_std:.4f}")

logger.info("\n=== АНАЛИЗ СТАБИЛЬНОСТИ ===")
# Анализируем, какая модель наиболее стабильна
for model_name, cv_metrics in cv_results.items():
    stability_score = 1.0 / (1.0 + cv_metrics["mae_std"])  # Чем меньше отклонение, тем выше стабильность
    logger.info(f"{model_name:15}: Индекс стабильности = {stability_score:.4f}")

logger.info("\n=== АНАЛИЗ ПЕРЕОБУЧЕНИЯ ===")
# Сравниваем результаты на обучающей и тестовой выборках для выявления переобучения
for model_name in ["RandomForest", "XGBoost"]:
    if model_name in classical_results:
        test_mae = classical_results[model_name]["mae"]
        # Для анализа переобучения используем CV результаты как приближение к обучающей выборке
        if model_name.lower() == "randomforest":
            cv_mae = cv_results.get("random_forest", {}).get("mae_mean", test_mae)
        elif model_name.lower() == "xgboost":
            cv_mae = cv_results.get("xgboost", {}).get("mae_mean", test_mae)
        else:
            cv_mae = test_mae

        overfitting_ratio = test_mae / max(cv_mae, 0.001)  # Избегаем деления на ноль
        if overfitting_ratio > 1.1:
            logger.info(f"{model_name:15}: Возможно переобучение (тест/CV: {overfitting_ratio:.3f})")
        else:
            logger.info(f"{model_name:15}: Переобучение не обнаружено (тест/CV: {overfitting_ratio:.3f})")

INFO:__main__:=== ОБОСНОВАНИЕ ВЫБОРА МОДЕЛЕЙ ===
INFO:__main__:1. Random Forest - ensemble-метод, устойчивый к переобучению, хорошо работает с молекулярными дескрипторами
INFO:__main__:2. XGBoost/Gradient Boosting - современный градиентный бустинг с высокой точностью
INFO:__main__:3. MLP - полносвязная нейросеть для изучения нелинейных паттернов
INFO:__main__:4. CNN - сверточная сеть для обнаружения локальных паттернов в дескрипторах
INFO:__main__:
=== ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА ОБУЧЕНИЯ МОДЕЛЕЙ ===

INFO:__main__:1. Обучение классических ensemble-моделей...
INFO:src.classical_models:Обучение Random Forest...
INFO:src.classical_models:Обучена модель RandomForest (n_estimators=100, max_depth=10) за 1.243 сек
INFO:src.classical_models:Обучение XGBoost/Gradient Boosting...
INFO:src.classical_models:Обучена модель XGBoost (n_estimators=100, max_depth=6, lr=0.1) за 0.687 сек
INFO:src.classical_models:Метрики для RandomForest: MAE=0.7560, RMSE=0.9649, R²=0.4618
INFO:src.classical_models:Метрик

In [5]:
# === ВИЗУАЛИЗАЦИЯ И ДЕТАЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ ===
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

logger.info("\n=== СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ ===")

# 1. Сравнение метрик моделей - барный график
model_names = list(all_results.keys())
mae_values = [all_results[name]["mae"] for name in model_names]
rmse_values = [all_results[name]["rmse"] for name in model_names]
r2_values = [all_results[name]["r2"] for name in model_names]
training_times = [all_results[name]["training_time"] for name in model_names]

# Создаем subplot с несколькими графиками
fig_metrics = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("MAE (ниже = лучше)", "RMSE (ниже = лучше)", "R² (выше = лучше)", "Время обучения (сек)"),
    specs=[[{"secondary_y": False}, {"secondary_y": False}], [{"secondary_y": False}, {"secondary_y": False}]],
)

# MAE
fig_metrics.add_trace(go.Bar(x=model_names, y=mae_values, name="MAE", marker_color="lightblue"), row=1, col=1)

# RMSE
fig_metrics.add_trace(go.Bar(x=model_names, y=rmse_values, name="RMSE", marker_color="lightcoral"), row=1, col=2)

# R²
fig_metrics.add_trace(go.Bar(x=model_names, y=r2_values, name="R²", marker_color="lightgreen"), row=2, col=1)

# Время обучения
fig_metrics.add_trace(go.Bar(x=model_names, y=training_times, name="Время", marker_color="lightyellow"), row=2, col=2)

fig_metrics.update_layout(title_text="Сравнение производительности моделей", showlegend=False, height=600)

fig_metrics.show()

# 2. Анализ стабильности (если есть результаты CV)
if cv_results:
    cv_model_names = list(cv_results.keys())
    cv_mae_means = [cv_results[name]["mae_mean"] for name in cv_model_names]
    cv_mae_stds = [cv_results[name]["mae_std"] for name in cv_model_names]
    cv_r2_means = [cv_results[name]["r2_mean"] for name in cv_model_names]
    cv_r2_stds = [cv_results[name]["r2_std"] for name in cv_model_names]

    # График стабильности
    fig_stability = go.Figure()

    # MAE с error bars
    fig_stability.add_trace(
        go.Bar(
            x=cv_model_names,
            y=cv_mae_means,
            error_y={"type": "data", "array": cv_mae_stds},
            name="MAE ± std",
            marker_color="orange",
        )
    )

    fig_stability.update_layout(
        title="Стабильность моделей (Кросс-валидация)",
        xaxis_title="Модель",
        yaxis_title="MAE ± стандартное отклонение",
        height=400,
    )

    fig_stability.show()

# 3. Интерпретируемость моделей (оценка важности признаков для Random Forest)
logger.info("\n=== АНАЛИЗ ИНТЕРПРЕТИРУЕМОСТИ ===")

# Получаем важность признаков для Random Forest
from src.classical_models import ClassicalModels

models_handler = ClassicalModels(random_state=42)
models_handler.train_random_forest(X_train, y_train, n_estimators=100)

# Получаем модель Random Forest
rf_model = models_handler.models["RandomForest"]["model"]
feature_importance = rf_model.feature_importances_
feature_names = X_train.columns

# Топ-20 наиболее важных признаков
top_indices = feature_importance.argsort()[-20:][::-1]
top_features = [feature_names[i] for i in top_indices]
top_importance = [feature_importance[i] for i in top_indices]

# График важности признаков
fig_importance = go.Figure(go.Bar(x=top_importance, y=top_features, orientation="h", marker_color="purple"))

fig_importance.update_layout(
    title="Топ-20 наиболее важных признаков (Random Forest)",
    xaxis_title="Важность признака",
    yaxis_title="Название признака",
    height=600,
)

fig_importance.show()

logger.info(f"Наиболее важные признаки: {', '.join(top_features[:5])}")

# 4. Анализ чувствительности к дескрипторам
logger.info("\n=== АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К ДЕСКРИПТОРАМ ===")

# Создаем краткий анализ типов дескрипторов
descriptor_types = {
    "morgan": sum(1 for col in feature_names if "morgan" in col.lower()),
    "rdkit": sum(1 for col in feature_names if "rdkit" in col.lower() and "morgan" not in col.lower()),
    "fingerprint": sum(1 for col in feature_names if "fp" in col.lower() or "fingerprint" in col.lower()),
    "mordred": sum(1 for col in feature_names if "mordred" in col.lower()),
    "other": len(feature_names),
}

# Пересчитываем 'other' как остаток
descriptor_types["other"] = len(feature_names) - sum(v for k, v in descriptor_types.items() if k != "other")

logger.info("Распределение типов дескрипторов в датасете:")
for desc_type, count in descriptor_types.items():
    if count > 0:
        logger.info(f"  {desc_type.capitalize()}: {count} признаков ({count / len(feature_names) * 100:.1f}%)")

# 5. Выводы и рекомендации
logger.info("\n=== ВЫВОДЫ И РЕКОМЕНДАЦИИ ===")

# Находим лучшую модель по R²
best_model_name = max(all_results.keys(), key=lambda x: all_results[x]["r2"])
best_r2 = all_results[best_model_name]["r2"]

logger.info(f"Лучшая модель по R²: {best_model_name} (R² = {best_r2:.4f})")

# Находим самую быструю модель
fastest_model_name = min(all_results.keys(), key=lambda x: all_results[x]["training_time"])
fastest_time = all_results[fastest_model_name]["training_time"]

logger.info(f"Самая быстрая модель: {fastest_model_name} (время = {fastest_time:.3f} сек)")

# Анализ соотношения качество/скорость
logger.info("\nСоотношение качество/скорость:")
for model_name in all_results:
    r2 = all_results[model_name]["r2"]
    time = all_results[model_name]["training_time"]
    efficiency = r2 / max(time, 0.001)  # R² на секунду
    logger.info(f"  {model_name:15}: Эффективность = {efficiency:.4f} (R²/сек)")

logger.info("\nРекомендации:")
logger.info("1. Для максимальной точности используйте лучшую модель по R²")
logger.info("2. Для быстрого прототипирования используйте самую быструю модель")
logger.info("3. Для продакшена рассмотрите компромисс между качеством и скоростью")
logger.info("4. Проведите дополнительную настройку гиперпараметров для улучшения результатов")

INFO:__main__:
=== СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ ===


INFO:__main__:
=== АНАЛИЗ ИНТЕРПРЕТИРУЕМОСТИ ===
INFO:src.classical_models:Обучена модель RandomForest (n_estimators=100, max_depth=None) за 1.727 сек


INFO:__main__:Наиболее важные признаки: padel_SpMax2_Bhm, rdkit_PEOE_VSA2, mordred_BCUTZ-1l, rdkit_TPSA, padel_BCUTp-1h
INFO:__main__:
=== АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К ДЕСКРИПТОРАМ ===
INFO:__main__:Распределение типов дескрипторов в датасете:
INFO:__main__:  Morgan: 398 признаков (59.1%)
INFO:__main__:  Rdkit: 46 признаков (6.8%)
INFO:__main__:  Fingerprint: 1 признаков (0.1%)
INFO:__main__:  Mordred: 91 признаков (13.5%)
INFO:__main__:  Other: 138 признаков (20.5%)
INFO:__main__:
=== ВЫВОДЫ И РЕКОМЕНДАЦИИ ===
INFO:__main__:Лучшая модель по R²: XGBoost (R² = 0.5231)
INFO:__main__:Самая быстрая модель: XGBoost (время = 0.687 сек)
INFO:__main__:
Соотношение качество/скорость:
INFO:__main__:  RandomForest   : Эффективность = 0.3715 (R²/сек)
INFO:__main__:  XGBoost        : Эффективность = 0.7618 (R²/сек)
INFO:__main__:  MLP            : Эффективность = 0.0699 (R²/сек)
INFO:__main__:  CNN            : Эффективность = 0.0317 (R²/сек)
INFO:__main__:
Рекомендации:
INFO:__main__:1. Для максималь

In [6]:
# === СОХРАНЕНИЕ РЕЗУЛЬТАТОВ В ФАЙЛЫ ===
from datetime import datetime
from pathlib import Path

logger.info("\n=== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===")

# Создаем директорию для результатов
results_dir = Path("data/model_results")
results_dir.mkdir(exist_ok=True)

# 1. Сохраняем основные результаты в JSON
results_json = {
    "test_results": all_results,
    "cross_validation_results": cv_results if cv_results else {},
    "feature_importance": {
        "top_features": top_features[:10],  # Топ-10 признаков
        "importance_values": [float(x) for x in top_importance[:10]],
    },
    "descriptor_analysis": descriptor_types,
    "summary": {
        "best_model_by_r2": {"name": best_model_name, "r2": best_r2},
        "fastest_model": {"name": fastest_model_name, "time": fastest_time},
        "total_features": len(feature_names),
        "training_date": datetime.now().isoformat(),
    },
}

json_path = results_dir / "model_training_results.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(results_json, f, indent=2, ensure_ascii=False)

logger.info(f"Результаты сохранены в JSON: {json_path}")

# 2. Сохраняем детальные метрики в CSV используя polars
test_metrics_data = []
for model_name, metrics in all_results.items():
    test_metrics_data.append(
        {
            "model_name": model_name,
            "mae": metrics["mae"],
            "mse": metrics["mse"],
            "rmse": metrics["rmse"],
            "r2": metrics["r2"],
            "training_time": metrics["training_time"],
        }
    )

test_metrics_df = pl.DataFrame(test_metrics_data)
csv_path = results_dir / "test_metrics.csv"
test_metrics_df.write_csv(csv_path)

logger.info(f"Метрики на тестовой выборке сохранены: {csv_path}")

# 3. Сохраняем результаты кросс-валидации в CSV
if cv_results:
    cv_metrics_data = []
    for model_name, cv_metrics in cv_results.items():
        cv_metrics_data.append(
            {
                "model_name": model_name,
                "mae_mean": cv_metrics["mae_mean"],
                "mae_std": cv_metrics["mae_std"],
                "rmse_mean": cv_metrics["rmse_mean"],
                "rmse_std": cv_metrics["rmse_std"],
                "r2_mean": cv_metrics["r2_mean"],
                "r2_std": cv_metrics["r2_std"],
            }
        )

    cv_metrics_df = pl.DataFrame(cv_metrics_data)
    cv_csv_path = results_dir / "cross_validation_metrics.csv"
    cv_metrics_df.write_csv(cv_csv_path)

    logger.info(f"Результаты кросс-валидации сохранены: {cv_csv_path}")

# 4. Сохраняем важность признаков в CSV
feature_importance_data = []
for i, (feature, importance) in enumerate(zip(top_features, top_importance, strict=False)):
    feature_importance_data.append({"rank": i + 1, "feature_name": feature, "importance": importance})

feature_importance_df = pl.DataFrame(feature_importance_data)
importance_csv_path = results_dir / "feature_importance.csv"
feature_importance_df.write_csv(importance_csv_path)

logger.info(f"Важность признаков сохранена: {importance_csv_path}")

# 5. Создаем итоговый отчет в текстовом формате
report_path = results_dir / "training_report.txt"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("=== ОТЧЕТ ПО ОБУЧЕНИЮ МОДЕЛЕЙ ===\n")
    f.write(f"Дата обучения: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Количество признаков: {len(feature_names)}\n")
    f.write(f"Размер обучающей выборки: {len(X_train)}\n")
    f.write(f"Размер тестовой выборки: {len(X_test)}\n\n")

    f.write("=== РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ ===\n")
    for model_name, metrics in all_results.items():
        f.write(
            f"{model_name:15}: MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}, R²={metrics['r2']:.4f}, Время={metrics['training_time']:.2f}s\n"
        )

    if cv_results:
        f.write("\n=== РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ ===\n")
        for model_name, cv_metrics in cv_results.items():
            mae_mean = cv_metrics["mae_mean"]
            mae_std = cv_metrics["mae_std"]
            r2_mean = cv_metrics["r2_mean"]
            r2_std = cv_metrics["r2_std"]
            f.write(f"{model_name:15}: MAE={mae_mean:.4f}±{mae_std:.4f}, R²={r2_mean:.4f}±{r2_std:.4f}\n")

    f.write("\n=== ВЫВОДЫ ===\n")
    f.write(f"Лучшая модель по R²: {best_model_name} (R² = {best_r2:.4f})\n")
    f.write(f"Самая быстрая модель: {fastest_model_name} (время = {fastest_time:.3f} сек)\n")
    f.write(f"Топ-5 признаков: {', '.join(top_features[:5])}\n")

logger.info(f"Итоговый отчет сохранен: {report_path}")

# Показываем все сохраненные файлы
logger.info("\nСохраненные файлы:")
for file_path in results_dir.glob("*"):
    file_size = file_path.stat().st_size
    logger.info(f"  {file_path.name}: {file_size} байт")

logger.info(f"\nВсе файлы сохранены в директории: {results_dir.absolute()}")

INFO:__main__:
=== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===
INFO:__main__:Результаты сохранены в JSON: data/model_results/model_training_results.json
INFO:__main__:Метрики на тестовой выборке сохранены: data/model_results/test_metrics.csv
INFO:__main__:Результаты кросс-валидации сохранены: data/model_results/cross_validation_metrics.csv
INFO:__main__:Важность признаков сохранена: data/model_results/feature_importance.csv
INFO:__main__:Итоговый отчет сохранен: data/model_results/training_report.txt
INFO:__main__:
Сохраненные файлы:
INFO:__main__:  model_training_results.json: 4884 байт
INFO:__main__:  cross_validation_metrics.csv: 442 байт
INFO:__main__:  feature_importance.csv: 810 байт
INFO:__main__:  improved_models_results.json: 423673 байт
INFO:__main__:  training_report.txt: 1096 байт
INFO:__main__:  modern_models_final.json: 4310 байт
INFO:__main__:  test_metrics.csv: 447 байт
INFO:__main__:  optimized_models_results.json: 714 байт
INFO:__main__:  modern_models_final_fixed.json: 2132 байт
INFO

In [7]:
# Анализ результатов и создание визуализаций
logger.info("\n=== АНАЛИЗ И ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ ===")

# Создание рейтинга моделей
ranking = create_model_ranking(
    all_results,
    {},  # Пустой словарь CV для упрощения
    weights={"mae": 0.4, "rmse": 0.3, "r2": 0.2, "stability": 0.1},
)

logger.info("\nРЕЙТИНГ МОДЕЛЕЙ:")
logger.info(str(ranking.select(["model", "score", "mae", "rmse", "r2"])))

# Создание визуализации сравнения
try:
    fig = create_performance_comparison(all_results, metrics=["mae", "rmse", "r2"], title="Сравнение производительности моделей")
    logger.info("\nГрафик сравнения создан успешно!")
    # fig.show()  # Раскомментировать для отображения
except Exception as e:
    logger.warning(f"\nОшибка создания графика (возможно, plotly не установлен): {e}")

logger.info("\n=== ВЫВОДЫ ===")
best_model = ranking.row(0)[0]
best_metrics = all_results[best_model]
logger.info(f"Лучшая модель: {best_model}")
logger.info(f"   MAE: {best_metrics['mae']:.4f}")
logger.info(f"   RMSE: {best_metrics['rmse']:.4f}")
logger.info(f"   R²: {best_metrics['r2']:.4f}")
logger.info(f"   Время обучения: {best_metrics['training_time']:.2f}s")

INFO:__main__:
=== АНАЛИЗ И ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ ===
INFO:src.model_comparison:Рейтинг моделей создан, лучшая модель: XGBoost
INFO:__main__:
РЕЙТИНГ МОДЕЛЕЙ:
INFO:__main__:shape: (4, 5)
┌──────────────┬──────────┬──────────┬──────────┬──────────┐
│ model        ┆ score    ┆ mae      ┆ rmse     ┆ r2       │
│ ---          ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str          ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞══════════════╪══════════╪══════════╪══════════╪══════════╡
│ XGBoost      ┆ 0.436493 ┆ 0.693442 ┆ 0.908288 ┆ 0.523088 │
│ RandomForest ┆ 0.369062 ┆ 0.755974 ┆ 0.964879 ┆ 0.461808 │
│ MLP          ┆ 0.252891 ┆ 0.837016 ┆ 1.074456 ┆ 0.332626 │
│ CNN          ┆ 0.207723 ┆ 0.869382 ┆ 1.114664 ┆ 0.281744 │
└──────────────┴──────────┴──────────┴──────────┴──────────┘
INFO:__main__:
График сравнения создан успешно!
INFO:__main__:
=== ВЫВОДЫ ===
INFO:__main__:Лучшая модель: XGBoost
INFO:__main__:   MAE: 0.6934
INFO:__main__:   RMSE: 0.9083
INFO:__main__:   R²: 0.523